In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()

'C:\\Users\\aditi'

In [3]:
print("hi")

hi


In [4]:
reviews_df = pd.read_csv("Downloads/employeeFeedbackDataset.csv", encoding="utf-8")

In [5]:
reviews_df

,Employee ID,Employee Chats,Employee Feedback,Division/Organization,Location of Office
0,EMP001,I love how leadership listens to our concerns ...,This initiative looks great but execution is e...,Marketing,"Atlanta, GA"
1,EMP002,It's so frustrating when decisions are made wi...,Great concept but people will only use it if i...,Learning & Development,"Los Angeles, CA"
2,EMP003,"If leadership stopped micromanaging, people wo...",This would improve morale but only if leadersh...,Finance,"Boston, MA"
3,EMP004,We spend too much time on manual reports when ...,This is a great idea but we need to consider c...,Customer Support,"New York, NY"
4,EMP005,This company has given me incredible career gr...,This is an exciting idea! If we get the right ...,Finance,"Dallas, TX"
...,...,...,...,...,...
145,EMP146,"Every time I catch up on work, another pile la...",This is a great idea but we need to consider c...,Learning & Development,"Washington, D.C."
146,EMP147,The company surprised us with a bonus today! D...,"If this isn't executed well, it could just be ...",Corporate Benefits,"Dallas, TX"
147,EMP148,Had a random conversation with a coworker toda...,This would improve morale but only if leadersh...,Marketing,"Seattle, WA"
148,EMP149,"Just got back from a training session, and it ...",It will only work if employees are given time ...,Administration,"Seattle, WA"


In [6]:
from google.cloud import language

def analyze_text_sentiment(text: str) -> language.AnalyzeSentimentResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_sentiment(document=document)

def show_text_sentiment(response: language.AnalyzeSentimentResponse):
    import pandas as pd

    columns = ["score", "sentence"]
    data = [(s.sentiment.score, s.text.content) for s in response.sentences]
    df_sentence = pd.DataFrame(columns=columns, data=data)

    sentiment = response.document_sentiment
    columns = ["score", "magnitude", "language"]
    data = [(sentiment.score, sentiment.magnitude, response.language)]
    df_document = pd.DataFrame(columns=columns, data=data)

    format_args = dict(index=False, tablefmt="presto", floatfmt="+.1f")
    print(f"At sentence level:\n{df_sentence.to_markdown(**format_args)}")
    print()
    print(f"At document level:\n{df_document.to_markdown(**format_args)}")
    

In [7]:
# Input
text = """
Python is a very readable language, which makes it easy to understand and maintain code.
It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks.
One disadvantage is its speed: it's not as fast as some other programming languages.
"""

# Send a request to the API
analyze_sentiment_response = analyze_text_sentiment(text)

# Show the results
show_text_sentiment(analyze_sentiment_response)

At sentence level:
   score | sentence
---------+------------------------------------------------------------------------------------------
    +0.8 | Python is a very readable language, which makes it easy to understand and maintain code.
    +0.9 | It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks.
    -0.4 | One disadvantage is its speed: it's not as fast as some other programming languages.

At document level:
   score |   magnitude | language
---------+-------------+------------
    +0.4 |        +2.2 | en


In [8]:

from tqdm import tqdm


tqdm.pandas()


def get_sentiment_score(text):
    if pd.isna(text) or text.strip() == "":
        return None 
    
    response = analyze_text_sentiment(text)
    return response.document_sentiment.score 

reviews_df["chat_sentiment"] = reviews_df["Employee Chats"].progress_apply(get_sentiment_score)
reviews_df["feedback_sentiment"] = reviews_df["Employee Feedback"].progress_apply(get_sentiment_score)

print(reviews_df.head())

100%|██████████| 150/150 [05:14<00:00,  2.10s/it]

  Employee ID                                     Employee Chats  \
0      EMP001  I love how leadership listens to our concerns ...   
1      EMP002  It's so frustrating when decisions are made wi...   
2      EMP003  If leadership stopped micromanaging, people wo...   
3      EMP004  We spend too much time on manual reports when ...   
4      EMP005  This company has given me incredible career gr...   

                                   Employee Feedback   Division/Organization  \
0  This initiative looks great but execution is e...               Marketing   
1  Great concept but people will only use it if i...  Learning & Development   
2  This would improve morale but only if leadersh...                 Finance   
3  This is a great idea but we need to consider c...        Customer Support   
4  This is an exciting idea! If we get the right ...                 Finance   

  Location of Office  chat_sentiment  feedback_sentiment  
0        Atlanta, GA             0.9               

In [11]:
reviews_df.head()

,Employee ID,Employee Chats,Employee Feedback,Division/Organization,Location of Office,chat_sentiment,feedback_sentiment
0,EMP001,I love how leadership listens to our concerns ...,This initiative looks great but execution is e...,Marketing,"Atlanta, GA",0.9,0.0
1,EMP002,It's so frustrating when decisions are made wi...,Great concept but people will only use it if i...,Learning & Development,"Los Angeles, CA",-0.8,0.0
2,EMP003,"If leadership stopped micromanaging, people wo...",This would improve morale but only if leadersh...,Finance,"Boston, MA",-0.3,-0.4
3,EMP004,We spend too much time on manual reports when ...,This is a great idea but we need to consider c...,Customer Support,"New York, NY",0.0,0.0
4,EMP005,This company has given me incredible career gr...,This is an exciting idea! If we get the right ...,Finance,"Dallas, TX",0.9,0.5


In [14]:
reviews_df.to_csv("employee.csv", index=False)

In [17]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

reviews_df['chat_emotion'] = reviews_df['Employee Chats'].apply(lambda x: classifier(x)[0]['label'] if pd.notnull(x) else 'None')
reviews_df['feedback_emotion'] = reviews_df['Employee Feedback'].apply(lambda x: classifier(x)[0]['label'] if pd.notnull(x) else 'None')

print(reviews_df.head())


Device set to use cpu


  Employee ID                                     Employee Chats  \
0      EMP001  I love how leadership listens to our concerns ...   
1      EMP002  It's so frustrating when decisions are made wi...   
2      EMP003  If leadership stopped micromanaging, people wo...   
3      EMP004  We spend too much time on manual reports when ...   
4      EMP005  This company has given me incredible career gr...   

                                   Employee Feedback   Division/Organization  \
0  This initiative looks great but execution is e...               Marketing   
1  Great concept but people will only use it if i...  Learning & Development   
2  This would improve morale but only if leadersh...                 Finance   
3  This is a great idea but we need to consider c...        Customer Support   
4  This is an exciting idea! If we get the right ...                 Finance   

  Location of Office  chat_sentiment  feedback_sentiment chat_emotion  \
0        Atlanta, GA             0.9 

In [18]:
reviews_df.head()

,Employee ID,Employee Chats,Employee Feedback,Division/Organization,Location of Office,chat_sentiment,feedback_sentiment,chat_emotion,feedback_emotion
0,EMP001,I love how leadership listens to our concerns ...,This initiative looks great but execution is e...,Marketing,"Atlanta, GA",0.9,0.0,joy,neutral
1,EMP002,It's so frustrating when decisions are made wi...,Great concept but people will only use it if i...,Learning & Development,"Los Angeles, CA",-0.8,0.0,sadness,neutral
2,EMP003,"If leadership stopped micromanaging, people wo...",This would improve morale but only if leadersh...,Finance,"Boston, MA",-0.3,-0.4,neutral,neutral
3,EMP004,We spend too much time on manual reports when ...,This is a great idea but we need to consider c...,Customer Support,"New York, NY",0.0,0.0,neutral,neutral
4,EMP005,This company has given me incredible career gr...,This is an exciting idea! If we get the right ...,Finance,"Dallas, TX",0.9,0.5,joy,joy


In [21]:
new_df = reviews_df

In [22]:
new_df.head()

,Employee ID,Employee Chats,Employee Feedback,Division/Organization,Location of Office,chat_sentiment,feedback_sentiment,chat_emotion,feedback_emotion
0,EMP001,I love how leadership listens to our concerns ...,This initiative looks great but execution is e...,Marketing,"Atlanta, GA",0.9,0.0,joy,neutral
1,EMP002,It's so frustrating when decisions are made wi...,Great concept but people will only use it if i...,Learning & Development,"Los Angeles, CA",-0.8,0.0,sadness,neutral
2,EMP003,"If leadership stopped micromanaging, people wo...",This would improve morale but only if leadersh...,Finance,"Boston, MA",-0.3,-0.4,neutral,neutral
3,EMP004,We spend too much time on manual reports when ...,This is a great idea but we need to consider c...,Customer Support,"New York, NY",0.0,0.0,neutral,neutral
4,EMP005,This company has given me incredible career gr...,This is an exciting idea! If we get the right ...,Finance,"Dallas, TX",0.9,0.5,joy,joy


In [23]:
reviews_df.to_csv("finalEmployee.csv", index=False)